In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
downloaded = drive.CreateFile({'id':'1Blpp94I9dOaMt7V4nB-0meR4FygacAwN'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Real_combine.csv') 

In [5]:
import pandas as pd


In [6]:
df=pd.read_csv('Real_combine.csv')

In [7]:
df

,Unnamed: 0,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,1,26.7,33.0,20.0,1012.4,60.0,5.1,4.4,13.0,284.795833
1,3,29.1,35.0,20.5,1011.9,49.0,5.8,5.2,14.8,219.720833
2,5,28.4,36.0,21.0,1011.3,46.0,5.3,5.7,11.1,182.187500
3,7,25.9,32.0,20.0,1011.8,56.0,6.1,6.9,11.1,154.037500
4,9,24.8,31.1,20.6,1013.6,58.0,4.8,8.3,11.1,223.208333
...,...,...,...,...,...,...,...,...,...,...
638,1277,28.5,33.4,20.9,1012.6,59.0,5.3,6.3,14.8,185.500000
639,1279,24.9,33.2,14.8,1011.5,48.0,4.2,4.6,13.0,166.875000
640,1281,26.4,32.0,20.9,1011.2,70.0,3.9,6.7,9.4,200.333333
641,1283,20.8,25.0,14.5,1016.8,78.0,4.7,5.9,11.1,349.291667


In [8]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [9]:
df

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,26.7,33.0,20.0,1012.4,60.0,5.1,4.4,13.0,284.795833
1,29.1,35.0,20.5,1011.9,49.0,5.8,5.2,14.8,219.720833
2,28.4,36.0,21.0,1011.3,46.0,5.3,5.7,11.1,182.187500
3,25.9,32.0,20.0,1011.8,56.0,6.1,6.9,11.1,154.037500
4,24.8,31.1,20.6,1013.6,58.0,4.8,8.3,11.1,223.208333
...,...,...,...,...,...,...,...,...,...
638,28.5,33.4,20.9,1012.6,59.0,5.3,6.3,14.8,185.500000
639,24.9,33.2,14.8,1011.5,48.0,4.2,4.6,13.0,166.875000
640,26.4,32.0,20.9,1011.2,70.0,3.9,6.7,9.4,200.333333
641,20.8,25.0,14.5,1016.8,78.0,4.7,5.9,11.1,349.291667


In [59]:
!pip install tpot

     |████████████████████████████████| 92kB 2.3MB/s 
     |████████████████████████████████| 163kB 8.1MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=12e46418b631250d02078013e2385809587007e2ad253511381977eb09d418cd
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [60]:
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [61]:
# Hyperparameter ranges

import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['mse','mae']}


In [62]:

X=df.iloc[:,:-1]
y=df.iloc[:,-1]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)




In [66]:
from tpot import TPOTRegressor


tpot_regressor = TPOTRegressor(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestRegressor': param}, 
                                 cv = 4)
tpot_regressor.fit(X_train,y_train)


Generation 1 - Current best internal CV score: -3336.187992566079
Generation 2 - Current best internal CV score: -3336.187992566079
Generation 3 - Current best internal CV score: -3335.2500905529214
Generation 4 - Current best internal CV score: -3335.2500905529214

Best pipeline: RandomForestRegressor(input_matrix, criterion=mse, max_depth=230, max_features=sqrt, min_samples_leaf=8, min_samples_split=10, n_estimators=400)


TPOTRegressor(config_dict={'sklearn.ensemble.RandomForestRegressor': {'criterion': ['mse',
                                                                                    'mae'],
                                                                      'max_depth': [10,
                                                                                    120,
                                                                                    230,
                                                                                    340,
                                                                                    450,
                                                                                    560,
                                                                                    670,
                                                                                    780,
                                                                                    890,
                 

In [68]:

score = tpot_regressor.score(X_test, y_test)
print(score)

-3527.209616942385


In [70]:
y_prediction=tpot_regressor.predict(X_test)

In [72]:
sklearn.metrics.mean_squared_error(y_test,y_prediction)

3527.209616942385